### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [4]:
# read the data into pandas dataframes
MS_T00_GD_210a = pd.read_csv("lca/cleaned/210_MS_T00_GD_lca.csv")

# delete any Metazoa hits because they're trypsin
MS_T00_GD_210b = MS_T00_GD_210a[MS_T00_GD_210a.kingdom != 'Metazoa']

# keep only entries to the phylum level
MS_T00_GD_210 = MS_T00_GD_210b[MS_T00_GD_210b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(MS_T00_GD_210))

MS_T00_GD_210.head(6)

# of phylum peptides =  33


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
6,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,QEFLNAAK,Phorcysia thermohydrogeniphila,Bacteria,NaN,NaN,NaN,Aquificae,NaN,NaN,Aquificae,...,Phorcysia,NaN,NaN,NaN,Phorcysia thermohydrogeniphila,NaN,NaN,NaN,NaN,NaN
64,LLSGMNQLK,Clostridium,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,Clostridium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,MTNGNFK,Moraxella,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,Moraxella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,VGHWLPR,Streptoalloteichus hindustanus,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,...,Streptoalloteichus,NaN,NaN,NaN,Streptoalloteichus hindustanus,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [5]:
MS_T00_GD_210_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/210_MS_T00_GD_DN50_ILnaafs.csv")
MS_T00_GD_210_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/210B_MS_T00_GD_DN50_ILnaafs.csv")

frames = [MS_T00_GD_210_NAAFa, MS_T00_GD_210_NAAFb]

MS_T00_GD_210_NAAF = pd.concat(frames, sort=False)

MS_T00_GD_210_NAAF.set_index('stripped_peptide')
MS_T00_GD_210_NAAF = MS_T00_GD_210_NAAF.loc[:, ~MS_T00_GD_210_NAAF.columns.str.contains('^Unnamed')]

MS_T00_GD_210_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(MS_T00_GD_210_NAAF))

print('column names:', MS_T00_GD_210_NAAF.columns)

MS_T00_GD_210_NAAF.head()

# of total peptides =  647
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,VLEGNEQFLNAAK,3160000.0,2.430769e+05
1,EGNEQFLNAAK,1570000.0,1.427273e+05
2,KMLNAAK,365000.0,5.214286e+04
3,KMLNAAK,365000.0,5.214286e+04
4,LSSPATLNSR,82100000.0,8.210000e+06


In [6]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(MS_T00_GD_210["peptide"], MS_T00_GD_210_NAAF["peptide"])

print(over)

['APVLSMR' 'EFHPGLAR' 'EFHPLQR' 'FAVWLPR' 'LATVLSPR' 'LLSGMNQLK'
 'LVGQNEAVNAVSNALR' 'NPWHDK' 'QEFLNAAK' 'SPGVSLPR' 'TEDAFVSK' 'TEEAVLELR'
 'TTTVSLPR' 'VATVSPLR' 'VAVVEAPR' 'VGHWLPR' 'VLGQNEAVNAVSNALR' 'VVTVLDPR'
 'VVTVVTPR']


In [7]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

MS_T00_GD_210.set_index('peptide', inplace=True)
MS_T00_GD_210_NAAF.set_index('peptide', inplace=True)

MS_T00_GD_210_Phy = MS_T00_GD_210.join(MS_T00_GD_210_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(MS_T00_GD_210_Phy))

MS_T00_GD_210_Phy.head()

# of total phylum-level peptides =  60


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
AENGLQPLK,Jeotgalibacillus,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Bacilli,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALDLLDEETR,Cladophialophora,Eukaryota,Fungi,Dikarya,NaN,Ascomycota,Pezizomycotina,NaN,Eurotiomycetes,Chaetothyriomycetidae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APVLSMR,Brevundimonas,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44200.0,6314.285714
ATTLGCK,Phaeocystis,Eukaryota,NaN,NaN,NaN,Haptista,NaN,NaN,Haptophyta,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AVTLLPSR,Microbacterium,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# write to a csv

MS_T00_GD_210_Phy.to_csv("lca/NAAF/MS_T00_GD_210_Phy_naaf.csv")